In [55]:
import re
import datetime
from enum import Enum

In [70]:
ENTRY_PATTERN = re.compile(r'\[(?P<time>.+)\] (?P<action>.+)')
BEGIN_PATTERN = re.compile(r'Guard #(?P<id>\d+) begins shift')
DATE_PATTERN = '%Y-%m-%d %H-%M'

class Action(Enum):
    START = 0
    END   = 1
    WAKE  = 2
    SLEEP = 3

In [71]:
def read_guard_history(filepath, pattern=ENTRY_PATTERN):
    with open(filepath) as file:
        for line in file.readlines():
            matches = pattern.match(line)
            if not matches:
                print(line)
                raise ValueError('Didnt find regex match')
            matches = matches.groupdict()
            date_str = matches['time']
            date = datetime.datetime.strptime(date_str, DATE_PATTERN)
            action = matches['action']
            yield (date, action)

In [50]:
filepath = 'data/day4.txt'

In [61]:
guard_history = list(read_guard_history(filepath))

In [74]:
def process_guard_history(guard_history):
    guard_history = sorted(guard_history, key=lambda x: x[0])
    guards = {}
    current_guard_id = None
    for entry in guard_history:
        date, action = entry
        # start shift/end shift
        match = BEGIN_PATTERN.match(action)
        if match:
            match = match.groupdict()
            if current_guard_id:
                guards[current_guard_id].append((date, Action.END))
            current_guard_id = int(match['id'])
            if current_guard_id not in guards:
                guards[current_guard_id] = [(date, Action.START)]
            continue
        # if guard id hasnt been set with non shift start
        if not current_guard_id:
            raise ValueError('Guard history doesnt start with guard shift')
        if current_guard_id not in guards:
            raise ValueError('Guard #{} hasnt started shift'.format(current_guard_id))
        # fall asleep
        if action == 'falls asleep':
            guards[current_guard_id].append((date, Action.SLEEP))
        elif action == 'wakes up':
            guards[current_guard_id].append((date, Action.WAKE))
        else:
            raise ValueError('Unknown action: [{}] Guard #{} {}'\
                .format(datr.strfmt(DATE_PATTERN), current_guard_id, action))
    return guards

In [76]:
guards = process_guard_history(guard_history)

In [94]:
def get_total_sleep_times(guards):
    sleep_time_minutes = {}
    def log_sleep_times_minutes(id, start, end):
        delta = end-start
        delta_minutes = int(delta.seconds / 60)
        for i in range(delta_minutes):
            curr_time = start+datetime.timedelta(minutes=i)
            curr_minute = curr_time.minute
            if curr_minute not in sleep_time_minutes[id]:
                sleep_time_minutes[id][curr_minute] = 0
            sleep_time_minutes[id][curr_minute] += 1
            
    sleep_times = {}
    def log_sleep_times(id, start, end):
        sleep_times[id] += (end-start).seconds / 60
        
    awake_times = {}
    def log_awake_times(id, start, end):
        awake_times[id] += (end-start).seconds / 60
        
    for id, actions in guards.items():
        sleep_time_minutes[id] = {}
        sleep_times[id] = 0
        awake_times[id] = 0
        prev_awake = None
        prev_sleep = None
        for date, action in actions:
            # if waking
            if action == Action.START or action == Action.WAKE:
                prev_awake = date
                if prev_sleep:
                    log_sleep_times_minutes(id, prev_sleep, date)
                    log_sleep_times(id, prev_sleep, date)
            # if sleeping
            elif action == Action.END or action == Action.SLEEP:
                prev_sleep = date
                if prev_awake:
                    log_sleep_times(id, prev_awake, date)
                    
    return (sleep_time_minutes, sleep_times, awake_times)

In [98]:
sleep_times_minutes, sleep_times, awake_times = get_total_sleep_times(guards)
id, total_minutes = max(sleep_times.items(), key=lambda x: x[1])
most_sleepy_minute, total_minutes = max(sleep_times_minutes[id].items(), key=lambda x: x[1])
checksum = id * most_sleepy_minute
print(checksum)

115167


In [100]:
# get (id, total_minutes) for each minute
# only keep the highest total minutes for each minute
most_frequently_slept_minutes = {}
for id, minute_sleep in sleep_times_minutes.items():
    for minute, total_minutes in minute_sleep.items():
        if minute not in most_frequently_slept_minutes:
            most_frequently_slept_minutes[minute] = (id, total_minutes)
        else:
            _, prev_max = most_frequently_slept_minutes[minute]
            if total_minutes > prev_max:
                most_frequently_slept_minutes[minute] = (id, total_minutes)

minute, (id, total_minutes) = max(most_frequently_slept_minutes.items(),
                                  key=lambda x: x[1][1])
checksum = id * minute
print(checksum)

32070
